# Determining Video Game Sales with regression Model (*incomplete)

In [1]:
#import libraries
import tensorflow as tf
from tensorflow import keras
import xgboost as xgb

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error, mean_absolute_error, accuracy_score, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV

In [2]:
# Import Data
df = pd.read_csv('drive/My Drive/z_Video_Game_Sales/Video_Games_Sales.csv')

In [3]:
print(df.shape)
df.head(3)

(16719, 16)


,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E


In [4]:
df.isna().sum()

Name                  2
Platform              0
Year_of_Release     269
Genre                 2
Publisher            54
NA_Sales              0
EU_Sales              0
JP_Sales              0
Other_Sales           0
Global_Sales          0
Critic_Score       8582
Critic_Count       8582
User_Score         6704
User_Count         9129
Developer          6623
Rating             6769
dtype: int64

In [5]:
df_temp = df.copy()

In [6]:
def fill_na(df_temp):
  # Fill NaN rows with the median
  for label, content in df_temp.items():
    if pd.api.types.is_numeric_dtype(content):
      if pd.isnull(content).sum():
        # Fill missing data with median
        df_temp[label] = content.fillna(content.median())
  return df_temp

In [7]:
def turn_num(df_temp):
  # Make all columns into numbers
  for label, content in df_temp.items():
    if not pd.api.types.is_numeric_dtype(content):
      # Add the +1 because pandas encodes missing categories as -1
      df_temp[label] = pd.Categorical(content).codes+1
  return df_temp   

In [8]:
# Check to see if data was correctly changed
df_temp = fill_na(df_temp)
df_temp = turn_num(df_temp)
df_temp.head(3)

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,11076,27,2006.0,11,362,41.36,28.96,3.77,8.45,82.53,76.0,51.0,78,322.0,1021,2
1,9390,12,1985.0,5,362,29.08,3.58,6.81,0.77,40.24,71.0,21.0,0,24.0,0,0
2,5614,27,2008.0,7,362,15.68,12.76,3.79,3.29,35.52,82.0,73.0,81,709.0,1021,2


In [9]:
# Develop X and y for model
X = df_temp.drop(['NA_Sales','EU_Sales','JP_Sales','Other_Sales','Global_Sales'], axis=1)
y = df_temp['Global_Sales']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Fit, Develop, and Test Model

In [10]:
model = RandomForestRegressor()
model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [11]:
model.score(X_test, y_test)

0.598280453362641

# Testing XGBoost model after Sub-optimal performance with RandomForestRegressor

In [12]:
# Setup 
df_xgb = df.copy()
df_xgb = turn_num(df_xgb)

X = df_xgb.drop(['NA_Sales','EU_Sales','JP_Sales','Other_Sales','Global_Sales'], axis=1)
y = df_xgb['Global_Sales']
# data_xgb = xgb.DMatrix(data=X,label=y)

In [13]:
X.head(3)

,Name,Platform,Year_of_Release,Genre,Publisher,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,11076,27,2006.0,11,362,76.0,51.0,78,322.0,1021,2
1,9390,12,1985.0,5,362,NaN,NaN,0,NaN,0,0
2,5614,27,2008.0,7,362,82.0,73.0,81,709.0,1021,2


In [14]:
# Setup Data and Model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
xg_model = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1, max_depth = 5, alpha = 10, n_estimators = 10)

In [15]:
# Fit Model and Train Data
xg_model.fit(X_train, y_train)
preds = xg_model.predict(X_test)

[22:14:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [16]:
xg_model.score(X_test, y_test)

0.11629732657197223

Filling data with mean, median, and leaving with NaN doesn't produce a functional model

In [17]:
# TODO: Adjust parameters to find good fitting model
# TODO: Determine what should be done with NaN values